In [1]:
mamba activate chrombpnet2

In [ ]:
cell_types=("Stem_cell" "Goblet_cell" "EEC")

parallel -j 3 '
echo "Processing {1}..." &&
samtools merge -f -@20 -o other_celltypes/{1}_human.bam \
    bam_per_cell_type_per_species/output_{1}@H9-tHIO-W15.5+4.bam \
    bam_per_cell_type_per_species/output_{1}@iPSC72.3-tHIO-W10.5.bam \
    bam_per_cell_type_per_species/output_{1}@JasonDuoCrypt-D80.bam \
    bam_per_cell_type_per_species/output_{1}@UmutSI-W17.bam &&
samtools sort -@10 other_celltypes/{1}_human.bam -o other_celltypes/{1}_human_sorted.bam &&
samtools index other_celltypes/{1}_human_sorted.bam &&
samtools view -b -@50 -F 780 -q 30 -F 1024 other_celltypes/{1}_human_sorted.bam > other_celltypes/{1}_human_sorted_filtered.bam &&
samtools index other_celltypes/{1}_human_sorted_filtered.bam
' ::: "${cell_types[@]}"


Processing Goblet_cell...
[bam_merge_core] Order of targets in file bam_per_cell_type_per_species/output_Goblet_cell@JasonDuoCrypt-D80.bam caused coordinate sort to be lost
[bam_merge_core] Order of targets in file bam_per_cell_type_per_species/output_Goblet_cell@UmutSI-W17.bam caused coordinate sort to be lost
[bam_sort_core] merging from 3 files and 10 in-memory blocks...
Processing EEC...
[bam_merge_core] Order of targets in file bam_per_cell_type_per_species/output_EEC@JasonDuoCrypt-D80.bam caused coordinate sort to be lost
[bam_merge_core] Order of targets in file bam_per_cell_type_per_species/output_EEC@UmutSI-W17.bam caused coordinate sort to be lost
[bam_sort_core] merging from 5 files and 10 in-memory blocks...


In [9]:
samtools flagstat enterocytes_species/enterocyte_human_sorted_filtered.bam

84243784 + 0 in total (QC-passed reads + QC-failed reads)
84243784 + 0 primary
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
84243784 + 0 mapped (100.00% : N/A)
84243784 + 0 primary mapped (100.00% : N/A)
84243784 + 0 paired in sequencing
42155604 + 0 read1
42088180 + 0 read2
83783032 + 0 properly paired (99.45% : N/A)
84243784 + 0 with itself and mate mapped
0 + 0 singletons (0.00% : N/A)
148886 + 0 with mate mapped to a different chr
148886 + 0 with mate mapped to a different chr (mapQ>=5)
(chrombpnet) 


: 1

In [10]:
mamba activate macs3
which macs3

/cluster/project/treutlein/jjans/software/miniforge3/envs/macs3/bin/macs3


In [ ]:
cell_types=("Stem_cell" "Goblet_cell" "EEC")

parallel -j 3 '
echo "Calling Peaks {1}..." &&
macs3 callpeak -f BAMPE -t other_celltypes/{1}_human_sorted_filtered.bam -g hs -n other_celltypes/called_peaks/{1}_human_peaks -B -q 0.01
' ::: "${cell_types[@]}"

In [15]:
mamba activate chrombpnet2
which bedtools

/cluster/project/treutlein/jjans/software/miniforge3/envs/chrombpnet2/bin/bedtools


In [16]:
cell_types=("Stem_cell" "Goblet_cell" "EEC")

parallel -j 3 '
bedtools intersect -v -a other_celltypes/called_peaks/{1}_human_peaks_peaks.narrowPeak -b encode_data/temp.bed  > other_celltypes/called_peaks/{1}_human_peaks_no_blacklist.bed &&
grep chr other_celltypes/called_peaks/{1}_human_peaks_no_blacklist.bed > other_celltypes/called_peaks/{1}_human_peaks_no_blacklist_chr.bed
' ::: "${cell_types[@]}"

***** WARNING: File other_celltypes/called_peaks/Goblet_cell_human_peaks_peaks.narrowPeak has inconsistent naming convention for record:
GL000008.2	2512	2834	other_celltypes/called_peaks/Goblet_cell_human_peaks_peak_1	52	.	5.11355	7.66683	5.21835	79

***** WARNING: File other_celltypes/called_peaks/Goblet_cell_human_peaks_peaks.narrowPeak has inconsistent naming convention for record:
GL000008.2	2512	2834	other_celltypes/called_peaks/Goblet_cell_human_peaks_peak_1	52	.	5.11355	7.66683	5.21835	79

***** WARNING: File other_celltypes/called_peaks/EEC_human_peaks_peaks.narrowPeak has inconsistent naming convention for record:
GL000195.1	30395	30876	other_celltypes/called_peaks/EEC_human_peaks_peak_1	87	.	6.90224	11.1865	8.79354	434

***** WARNING: File other_celltypes/called_peaks/EEC_human_peaks_peaks.narrowPeak has inconsistent naming convention for record:
GL000195.1	30395	30876	other_celltypes/called_peaks/EEC_human_peaks_peak_1	87	.	6.90224	11.1865	8.79354	434

***** WARNING: File ot

In [12]:
wc -l encode_data/temp.bed

910 encode_data/temp.bed


In [17]:
mamba activate chrombpnet2
which chrombpnet

/cluster/project/treutlein/jjans/software/miniforge3/envs/chrombpnet2/bin/chrombpnet


In [ ]:
cell_types=("Stem_cell" "Goblet_cell" "EEC")
folds=("fold_1" "fold_2" "fold_3" "fold_4" "fold_0")

# Function to create output directory if it doesn't exist
create_output_dir() {
    local output_dir="$1"
    mkdir -p $output_dir
}

# Export the function so that GNU Parallel can access it
export -f create_output_dir


parallel -j 6 '
    output_dir="other_celltypes/modelv1_{1}/{2}/output"
    create_output_dir "$output_dir"
    chrombpnet prep nonpeaks -g encode_data/hg38.fa \
        -p other_celltypes/called_peaks/{1}_human_peaks_no_blacklist_chr.bed \
        -c encode_data/hg38.chrom.sizes \
        -fl encode_data/splits/{2}.json \
        -br encode_data/blacklist.bed.gz \
        -o other_celltypes/modelv1_{1}/{2}/output
' ::: "${cell_types[@]}" ::: "${folds[@]}"


num_rows:39507
Number of regions filtered because inputlen sequence cannot be constructed: 6
Percentage of regions filtered 0.015%
Following background chromosomes chrUn_KI270375v1,chrUn_KI270750v1,chr1_KI270710v1_random,chrUn_KI270589v1,chr14_KI270724v1_random,chrUn_KI270744v1,chr17_KI270730v1_random,chrUn_KI270751v1,chr1_KI270708v1_random,chrUn_KI270315v1,chrUn_KI270420v1,chr14_GL000225v1_random,chr22_KI270733v1_random,chr1_KI270709v1_random,chr14_GL000009v2_random,chr5_GL000208v1_random,chr22_KI270738v1_random,chrUn_KI270442v1,chrUn_KI270741v1,chr2_KI270716v1_random,chr9_KI270718v1_random,chr1_KI270711v1_random,chr22_KI270734v1_random,chrUn_KI270302v1,chr14_KI270725v1_random,chr3_GL000221v1_random,chrUn_KI270517v1,chrUn_KI270754v1,chr1_KI270707v1_random,chr15_KI270727v1_random,chrUn_GL000219v1,chr9_KI270719v1_random,chrUn_KI270591v1,chrUn_KI270755v1,chrUn_KI270746v1,chr14_KI270723v1_random,chrUn_KI270587v1,chrUn_KI270448v1,chrUn_GL000195v1,chrEBV,chrUn_KI270584v1,chr14_KI270726v1_ra

In [29]:
cell_types=("Stem_cell" "Goblet_cell" "EEC")
folds=("fold_1" "fold_2" "fold_3" "fold_4" "fold_0")

# Function to create output directory if it doesn't exist
create_output_dir() {
    local output_dir="$1"
    mkdir -p $output_dir
}

# Export the function so that GNU Parallel can access it
export -f create_output_dir


parallel -j 6 '
    output_dir="/cluster/project/treutlein/USERS/jjans/analysis/intestine/other_celltypes/modelv1_{1}/{2}/output"
    create_output_dir "$output_dir"
    export bam_file=/cluster/project/treutlein/USERS/jjans/analysis/intestine/other_celltypes/{1}_human_sorted_filtered.bam
    export peaks=/cluster/project/treutlein/USERS/jjans/analysis/intestine/other_celltypes/called_peaks/{1}_human_peaks_no_blacklist_chr.bed
    export negative_peaks=/cluster/project/treutlein/USERS/jjans/analysis/intestine/other_celltypes/modelv1_{1}/{2}/output_negatives.bed
    export fold_json=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/splits/{2}.json
    export output_model=/cluster/project/treutlein/USERS/jjans/analysis/intestine/other_celltypes/modelv1_{1}/{2}/output
    export genome_fasta=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.fa
    export genome_sizes=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.chrom.sizes
    export bias_model=/cluster/project/treutlein/USERS/jjans/analysis/intestine/bias_model/scATAC_dermal_fibroblast.h5

    sbatch train_chrombpnet.sh "$bam_file" "$peaks" "$negative_peaks" "$fold_json" "$output_model" "$genome_fasta" "$genome_sizes" "$bias_model"
' ::: "${cell_types[@]}" ::: "${folds[@]}"


Submitted batch job 22802
Submitted batch job 22803
Submitted batch job 22804
Submitted batch job 22806
Submitted batch job 22805
Submitted batch job 22807
Submitted batch job 22808
Submitted batch job 22809
Submitted batch job 22810
Submitted batch job 22811
Submitted batch job 22812
Submitted batch job 22813
Submitted batch job 22814
Submitted batch job 22816
Submitted batch job 22815


In [41]:
ls modelv1*/chrombpnet_model/evaluation/overall_report.pdf

modelv1/chrombpnet_model/evaluation/overall_report.pdf
modelv1_fold1/chrombpnet_model/evaluation/overall_report.pdf
modelv1_fold2/chrombpnet_model/evaluation/overall_report.pdf
modelv1_fold3/chrombpnet_model/evaluation/overall_report.pdf


In [43]:
ls modelv1*/chrombpnet_model/models/chrombpnet_nobias.h5

modelv1/chrombpnet_model/models/chrombpnet_nobias.h5
modelv1_fold1/chrombpnet_model/models/chrombpnet_nobias.h5
modelv1_fold2/chrombpnet_model/models/chrombpnet_nobias.h5
modelv1_fold3/chrombpnet_model/models/chrombpnet_nobias.h5
modelv1_fold4/chrombpnet_model/models/chrombpnet_nobias.h5


In [44]:
ls modelv*chimp*/chrombpnet_model/models/chrombpnet_nobias.h5

modelv0_chimp/chrombpnet_model/models/chrombpnet_nobias.h5
modelv0_chimp_fold1/chrombpnet_model/models/chrombpnet_nobias.h5
modelv0_chimp_fold2/chrombpnet_model/models/chrombpnet_nobias.h5


In [42]:
ls modelv*chimp*/chrombpnet_model/evaluation/overall_report.pdf

modelv0_chimp/chrombpnet_model/evaluation/overall_report.pdf
modelv0_chimp_fold1/chrombpnet_model/evaluation/overall_report.pdf
modelv0_chimp_fold2/chrombpnet_model/evaluation/overall_report.pdf


In [1]:
mamba activate macs3
which macs3

/cluster/project/treutlein/jjans/software/miniforge3/envs/macs3/bin/macs3


In [7]:
export bam_file_enterocytes=/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/enterocytes_human_sorted_filtered.bam
export bam_file_EEC=/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/EEC_human_sorted_filtered.bam
export bam_file_Stem_cell=/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/Stem_cell_human_sorted_filtered.bam
export bam_file_Goblet_cell=/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/Goblet_cell_human_sorted_filtered.bam

macs3 callpeak -f BAMPE -t $bam_file_enterocytes -c $bam_file_EEC $bam_file_Stem_cell $bam_file_Goblet_cell -g hs -n celltype_models_human/diff_peaks/enterocytes_vs_rest.bam -B -q 0.01 &> celltype_models_human/diff_peaks/enterocytes_vs_rest.log
macs3 callpeak -f BAMPE -t $bam_file_EEC -c $bam_file_enterocytes $bam_file_Stem_cell $bam_file_Goblet_cell -g hs -n celltype_models_human/diff_peaks/EEC_vs_rest.bam -B -q 0.01 &> celltype_models_human/diff_peaks/EEC_vs_rest.log
macs3 callpeak -f BAMPE -t $bam_file_Stem_cell -c $bam_file_EEC $bam_file_enterocytes $bam_file_Goblet_cell -g hs -n celltype_models_human/diff_peaks/Stem_cell_vs_rest.bam -B -q 0.01 &> celltype_models_human/diff_peaks/Stem_cell_vs_rest.log
macs3 callpeak -f BAMPE -t $bam_file_Goblet_cell -c $bam_file_EEC $bam_file_Stem_cell $bam_file_enterocytes -g hs -n celltype_models_human/diff_peaks/Goblet_cell_vs_rest.bam -B -q 0.01 &> celltype_models_human/diff_peaks/Goblet_cell_vs_rest.log

In [4]:
ls celltype_models_human/diff_peaks

In [9]:
mamba activate chrombpnet2
which bedtools

/cluster/project/treutlein/jjans/software/miniforge3/envs/chrombpnet2/bin/bedtools


In [10]:
cell_types=("Stem_cell" "Goblet_cell" "EEC" "enterocytes")

parallel -j 3 '
bedtools intersect -v -a celltype_models_human/diff_peaks/{1}_vs_rest.bam_peaks.narrowPeak -b encode_data/temp.bed  > celltype_models_human/diff_peaks/{1}_vs_rest.bam_peaks_no_blacklist.bed &&
grep chr celltype_models_human/diff_peaks/{1}_vs_rest.bam_peaks_no_blacklist.bed > celltype_models_human/diff_peaks/{1}_vs_rest.bam_peaks_no_blacklist_chr.bed
' ::: "${cell_types[@]}"

***** WARNING: File celltype_models_human/diff_peaks/Goblet_cell_vs_rest.bam_peaks.narrowPeak has inconsistent naming convention for record:
KI270438.1	104558	104711	celltype_models_human/diff_peaks/Goblet_cell_vs_rest.bam_peak_1	41	.	2.15145	8.50341	4.16608	140

***** WARNING: File celltype_models_human/diff_peaks/Goblet_cell_vs_rest.bam_peaks.narrowPeak has inconsistent naming convention for record:
KI270438.1	104558	104711	celltype_models_human/diff_peaks/Goblet_cell_vs_rest.bam_peak_1	41	.	2.15145	8.50341	4.16608	140

***** WARNING: File celltype_models_human/diff_peaks/Stem_cell_vs_rest.bam_peaks.narrowPeak has inconsistent naming convention for record:
GL000205.2	63045	63435	celltype_models_human/diff_peaks/Stem_cell_vs_rest.bam_peak_1	32	.	2.14676	6.40941	3.2741	172

***** WARNING: File celltype_models_human/diff_peaks/Stem_cell_vs_rest.bam_peaks.narrowPeak has inconsistent naming convention for record:
GL000205.2	63045	63435	celltype_models_human/diff_peaks/Stem_cell_vs_rest.ba

In [13]:
ls celltype_models_human/diff_peaks/*_peaks_no_blacklist_chr.bed | xargs cat > celltype_models_human/diff_peaks/concat_no_blacklist_chr.bed

In [14]:
wc -l celltype_models_human/diff_peaks/concat_no_blacklist_chr.bed

60020 celltype_models_human/diff_peaks/concat_no_blacklist_chr.bed


In [15]:
ls celltype_models_human/diff_peaks/*_chr.bed

celltype_models_human/diff_peaks/EEC_vs_rest.bam_peaks_no_blacklist_chr.bed
celltype_models_human/diff_peaks/Goblet_cell_vs_rest.bam_peaks_no_blacklist_chr.bed
celltype_models_human/diff_peaks/Stem_cell_vs_rest.bam_peaks_no_blacklist_chr.bed
celltype_models_human/diff_peaks/concat_no_blacklist_chr.bed
celltype_models_human/diff_peaks/enterocytes_vs_rest.bam_peaks_no_blacklist_chr.bed


In [16]:
wc -l celltype_models_human/diff_peaks/EEC_vs_rest.bam_peaks_no_blacklist_chr.bed

27985 celltype_models_human/diff_peaks/EEC_vs_rest.bam_peaks_no_blacklist_chr.bed


In [27]:
bedtools sort -i celltype_models_human/diff_peaks/concat_no_blacklist_chr.bed > celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.bed

In [29]:
bedtools merge -i celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.bed > celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.merged.bed

In [30]:
wc -l celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.merged.bed

59395 celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.merged.bed


In [39]:
head celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.bed

chr1	897985	899212	celltype_models_human/diff_peaks/EEC_vs_rest.bam_peak_2	1046	.	31.891	111.367	104.625	902
chr1	906131	906359	celltype_models_human/diff_peaks/Stem_cell_vs_rest.bam_peak_11	38	.	2.43107	7.12789	3.89742	158
chr1	916788	917046	celltype_models_human/diff_peaks/Stem_cell_vs_rest.bam_peak_12	35	.	2.00238	6.68709	3.51577	6
chr1	935791	936012	celltype_models_human/diff_peaks/EEC_vs_rest.bam_peak_3	52	.	3.22633	8.11162	5.231	51
chr1	941620	942083	celltype_models_human/diff_peaks/EEC_vs_rest.bam_peak_4	27	.	1.85716	5.25817	2.70395	186
chr1	943097	943473	celltype_models_human/diff_peaks/EEC_vs_rest.bam_peak_5	121	.	4.30936	15.5946	12.1672	224
chr1	1031725	1032180	celltype_models_human/diff_peaks/enterocytes_vs_rest.bam_peak_26	48	.	2.04014	7.78408	4.80755	390
chr1	1073966	1074233	celltype_models_human/diff_peaks/Goblet_cell_vs_rest.bam_peak_3	21	.	1.70959	6.12686	2.17473	0
chr1	1079682	1079899	celltype_models_human/diff_peaks/Stem_cell_vs_rest.bam_peak_13	23	.	1.37603	5.29249	2

In [42]:
wc -l celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.bed

60020 celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.bed


In [44]:
ls /cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.bed

/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.bed


In [41]:
wc -l celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.merged.bed

59395 celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.merged.bed


In [1]:
mamba activate chrombpnet2

In [4]:
ls extra_data/bed_files

All_human_SNC.bed  DARs_Early_enterocyte.bed  DARs_Stemcell.bed  HAR.bed
CMS.bed		   DARs_Enterocyte.bed	      GWAS.bed		 SNC.bed
DARs_EEC.bed	   DARs_Gobletcell.bed	      HAQER.bed


In [6]:
regions=("All_human_SNC" "CMS" "DARs_EEC" "DARs_Early_enterocyte" "DARs_Enterocyte" "DARs_Gobletcell" "DARs_Stemcell" "GWAS" "HAQER" "HAR" "SNC")

parallel -j 3 '
bedtools intersect -a called_peaks/peaks_no_blacklist_chr.bed -b /cluster/project/treutlein/USERS/jjans/analysis/intestine/extra_data/bed_files/{1}.bed -wa > regions_of_interest/{1}.bed
' ::: "${regions[@]}"

In [9]:
head regions_of_interest/HAQER.bed

chr1	11907786	11909269	human_peaks_peak_723	622	.	6.15136	65.368	62.2506	322
chr1	39519381	39519754	human_peaks_peak_2250	340	.	12.6562	36.7916	34.0884	56
chr1	143729506	143729814	human_peaks_peak_5067	157	.	7.8348	18.0782	15.7334	261
chr1	144412132	144413331	human_peaks_peak_5071	925	.	14.0753	96.1379	92.5833	579
chr1	147928210	147928806	human_peaks_peak_5172	1289	.	17.053	133.159	128.963	183
chr1	148522174	148523180	human_peaks_peak_5201	1356	.	15.8512	139.95	135.609	548
chr1	157978770	157979303	human_peaks_peak_5717	66	.	4.62318	8.68645	6.62189	477
chr11	56394595	56394908	human_peaks_peak_14797	147	.	7.53346	17.0268	14.7077	199
chr11	71775086	71775614	human_peaks_peak_15870	50	.	4.09197	7.08138	5.08527	451
chr11	71786402	71787948	human_peaks_peak_15871	1618	.	8.20952	166.775	161.804	659


In [5]:
ls 

All_human_SNC.bed  DARs_Early_enterocyte.bed  DARs_Stemcell.bed  HAR.bed
CMS.bed		   DARs_Enterocyte.bed	      GWAS.bed		 SNC.bed
DARs_EEC.bed	   DARs_Gobletcell.bed	      HAQER.bed


In [ ]:
cell_types=("enterocytes" "Stem_cell" "Goblet_cell" "EEC")
folds=("fold_1" "fold_2" "fold_3" "fold_4" "fold_0")
regions=("All_human_SNC" "CMS" "DARs_EEC" "DARs_Early_enterocyte" "DARs_Enterocyte" "DARs_Gobletcell" "DARs_Stemcell" "GWAS" "HAQER" "HAR" "SNC")
# Function to create output directory if it doesn't exist
create_output_dir() {
    local output_dir="$1"
    mkdir -p $output_dir
}

# Export the function so that GNU Parallel can access it
export -f create_output_dir


parallel -j 6 '
    output_dir="/cluster/project/treutlein/USERS/jjans/analysis/intestine/region_scores/{1}/{3}/"
    create_output_dir "$output_dir"
    
    export model_path=/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/modelv1_{1}/{2}/output/models/chrombpnet.h5
    
    export regions=/cluster/project/treutlein/USERS/jjans/analysis/intestine/regions_of_interest/{3}.bed
    
    export output_prefix=/cluster/project/treutlein/USERS/jjans/analysis/intestine/region_scores/{1}/{3}/{2}
    
    export genome_fasta=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.fa
    export genome_sizes=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.chrom.sizes

    sbatch chrombpnet_contribs.sh "$model_path" "$regions" "$genome_fasta" "$genome_sizes" "$output_prefix"
' ::: "${cell_types[@]}" ::: "${folds[@]}" ::: "${regions[@]}" 



In [ ]:
cell_types=("enterocytes" "Stem_cell" "Goblet_cell" "EEC")
folds=("fold_1" "fold_2" "fold_3" "fold_4" "fold_0")

# Function to create output directory if it doesn't exist
create_output_dir() {
    local output_dir="$1"
    mkdir -p $output_dir
}

# Export the function so that GNU Parallel can access it
export -f create_output_dir


parallel -j 6 '
    output_dir="/cluster/project/treutlein/USERS/jjans/analysis/intestine/region_scores/{1}/diff_peaks/"
    create_output_dir "$output_dir"
    
    export model_path=/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/modelv1_{1}/{2}/output/models/chrombpnet.h5
    
    export regions=/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.bed
    
    export output_prefix=/cluster/project/treutlein/USERS/jjans/analysis/intestine/region_scores/{1}/diff_peaks/{2}
    
    export genome_fasta=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.fa
    export genome_sizes=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.chrom.sizes

    sbatch chrombpnet_contribs.sh "$model_path" "$regions" "$genome_fasta" "$genome_sizes" "$output_prefix"
' ::: "${cell_types[@]}" ::: "${folds[@]}"



In [45]:
cell_types=("enterocytes" "Stem_cell" "Goblet_cell" "EEC")
folds=("fold_1" "fold_2" "fold_3" "fold_4" "fold_0")

# Function to create output directory if it doesn't exist
create_output_dir() {
    local output_dir="$1"
    mkdir -p $output_dir
}

# Export the function so that GNU Parallel can access it
export -f create_output_dir


parallel -j 6 '
    output_dir="/cluster/project/treutlein/USERS/jjans/analysis/intestine/region_scores/{1}/diff_peaks/"
    create_output_dir "$output_dir"
    
    export model_path=/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/modelv1_{1}/{2}/output/models/chrombpnet.h5
    
    export regions=/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.bed
    
    export output_prefix=/cluster/project/treutlein/USERS/jjans/analysis/intestine/region_scores/{1}/diff_peaks/{2}
    
    export genome_fasta=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.fa
    export genome_sizes=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.chrom.sizes

    sbatch chrombpnet_contribs.sh "$model_path" "$regions" "$genome_fasta" "$genome_sizes" "$output_prefix"
' ::: "${cell_types[@]}" ::: "${folds[@]}"



Submitted batch job 83440
Submitted batch job 83441
Submitted batch job 83442
Submitted batch job 83443
Submitted batch job 83444
Submitted batch job 83445
Submitted batch job 83446
Submitted batch job 83447
Submitted batch job 83448
Submitted batch job 83449
Submitted batch job 83450
Submitted batch job 83451
Submitted batch job 83452
Submitted batch job 83453
Submitted batch job 83454
Submitted batch job 83455
Submitted batch job 83456
Submitted batch job 83457
Submitted batch job 83458
Submitted batch job 83459


In [37]:
ls region_scores/*

region_scores/EEC:
AIG_regions  SLC5A12_regions  diff_peaks

region_scores/Goblet_cell:
AIG_regions  SLC5A12_regions  diff_peaks

region_scores/Stem_cell:
AIG_regions  AIG_regions_peaks	SLC5A12_regions  diff_peaks

region_scores/enterocytes:
AIG_regions  SLC5A12_regions  diff_peaks


In [48]:
which python

/cluster/project/treutlein/jjans/software/miniforge3/envs/chrombpnet2/bin/python


In [49]:
which bamCoverage

/cluster/project/treutlein/jjans/software/miniforge3/envs/chrombpnet2/bin/bamCoverage


In [ ]:
cell_types=("EEC" "enterocytes" "Stem_cell" "Goblet_cell")

parallel -j 2 '
bamCoverage -b /cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/{1}_human_sorted_filtered.bam -o /cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/{1}_human_sorted_filtered.RPGC_norm.bw -p max/2 --normalizeUsing RPGC -bs 1 --effectiveGenomeSize 2913022398
' ::: "${cell_types[@]}"

normalization: 1x (effective genome size 2913022398)
bamFilesList: ['/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/EEC_human_sorted_filtered.bam']
binLength: 1
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 9
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 1
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000
normalization: 1x (effective genome size 2913022398)
bamFilesList: ['/cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/enterocytes_human_sorted_filtered.bam']
binLength: 1
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkS

In [24]:
head /cluster/project/treutlein/USERS/jjans/analysis/intestine/celltype_models_human/diff_peaks/concat_no_blacklist_chr.sorted.merged.bed

chr1	897985	899212
chr1	906131	906359
chr1	916788	917046
chr1	935791	936012
chr1	941620	942083
chr1	943097	943473
chr1	1031725	1032180
chr1	1073966	1074233
chr1	1079682	1079899
chr1	1127180	1127454


In [ ]:
macs3 callpeak -f BAMPE -t other_celltypes/{1}_human_sorted_filtered.bam -g hs -n other_celltypes/called_peaks/{1}_human_peaks -B -q 0.01


In [21]:
ls *.sh

chrombpnet_contribs.sh	train_chrombpnet.sh


In [ ]:
model_path="$1"
regions="$2"
genome_fasta="$3"
genome_sizes="$4"
output_prefix="$5"



In [ ]:
chrombpnet contribs_bw \
-m modelv0_chimp/chrombpnet_model/models/chrombpnet_nobias.h5 \
-r regions_of_interest/final_test_regions_peaks.bed \
-g encode_data/hg38.fa -c encode_data/hg38.chrom.sizes -op test_region_final_chimp


In [ ]:
# Define input variables
cell_types=("Stem_cell" "Goblet_cell" "EEC")
folds=("fold_1" "fold_2" "fold_3" "fold_4" "fold_0")

# Function to run chrombpnet contribs_bw
run_contribs() {
    local cell_type="$1"
    local fold="$2"
    
    local model_path="modelv0_${cell_type}/chrombpnet_model/models/chrombpnet_${fold}.h5"
    local regions="regions_of_interest/final_test_regions_peaks.bed"
    local genome_fasta=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.fa
    local genome_sizes=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.chrom.sizes

    local output_prefix="region_scores/${cell_type}/${fold}/final_test_regions"
    
    # Run chrombpnet contribs_bw command
    chrombpnet contribs_bw \
        -m "$model_path" \
        -r "$regions" \
        -g "$genome_fasta" \
        -c "$genome_sizes" \
        -op "$output_prefix"
}

# Export the function so that GNU Parallel can access it
export -f run_contribs

# Use GNU Parallel to iterate over combinations of cell types and folds
parallel -j 6 run_contribs {1} {2} ::: "${cell_types[@]}" ::: "${folds[@]}"


In [ ]:
# Define input variables
cell_types=("Stem_cell" "Goblet_cell" "EEC")
folds=("fold_1" "fold_2" "fold_3" "fold_4" "fold_0")

# Function to run chrombpnet contribs_bw
run_contribs() {
    local cell_type="$1"
    local fold="$2"
    
    local model_path="modelv0_${cell_type}/chrombpnet_model/models/chrombpnet_${fold}.h5"
    local regions="regions_of_interest/final_test_regions_peaks.bed"
    local genome_fasta=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.fa
    local genome_sizes=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.chrom.sizes

    local output_prefix="region_scores/${cell_type}/${fold}/final_test_regions"
    
    # Run chrombpnet contribs_bw command
    chrombpnet contribs_bw \
        -m "$model_path" \
        -r "$regions" \
        -g "$genome_fasta" \
        -c "$genome_sizes" \
        -op "$output_prefix"
}

# Export the function so that GNU Parallel can access it
export -f run_contribs

# Use GNU Parallel to iterate over combinations of cell types and folds
parallel -j 6 run_contribs {1} {2} ::: "${cell_types[@]}" ::: "${folds[@]}"


In [ ]:
cell_types=("Stem_cell" "Goblet_cell" "EEC")
folds=("fold_1" "fold_2" "fold_3" "fold_4" "fold_0")

# Function to create output directory if it doesn't exist
create_output_dir() {
    local output_dir="$1"
    mkdir -p $output_dir
}

# Export the function so that GNU Parallel can access it
export -f create_output_dir


parallel -j 6 '
    output_dir="/cluster/project/treutlein/USERS/jjans/analysis/intestine/other_celltypes/modelv1_{1}/{2}/output"
    create_output_dir "$output_dir"
    export bam_file=/cluster/project/treutlein/USERS/jjans/analysis/intestine/other_celltypes/{1}_human_sorted_filtered.bam
    export peaks=/cluster/project/treutlein/USERS/jjans/analysis/intestine/other_celltypes/called_peaks/{1}_human_peaks_no_blacklist_chr.bed
    export negative_peaks=/cluster/project/treutlein/USERS/jjans/analysis/intestine/other_celltypes/modelv1_{1}/{2}/output_negatives.bed
    export fold_json=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/splits/{2}.json
    export output_model=/cluster/project/treutlein/USERS/jjans/analysis/intestine/other_celltypes/modelv1_{1}/{2}/output
    export genome_fasta=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.fa
    export genome_sizes=/cluster/project/treutlein/USERS/jjans/analysis/intestine/encode_data/hg38.chrom.sizes
    export bias_model=/cluster/project/treutlein/USERS/jjans/analysis/intestine/bias_model/scATAC_dermal_fibroblast.h5

    sbatch train_chrombpnet.sh "$bam_file" "$peaks" "$negative_peaks" "$fold_json" "$output_model" "$genome_fasta" "$genome_sizes" "$bias_model"
' ::: "${cell_types[@]}" ::: "${folds[@]}"


In [27]:
pwd

/cluster/project/treutlein/USERS/jjans/analysis/intestine


In [ ]:
# call the submission script
sbatch train_chrombpnet.sh "$bam_file" "$peaks" "$negative_peaks" "$fold_json" "$output_model" "$genome_fasta" "$genome_sizes" "$bias_model"


In [ ]:
bam_file="$1"
peaks="$2"
negative_peaks="$3"
fold_json="$4"
output_model="$5"
genome_fasta="$6"
genome_sizes="$7"
bias_model="$8"

